<a href="https://colab.research.google.com/github/cgbas/tc-data-analytics/blob/feature%2Ffase2/fase%202/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo preditivo Ibovespa

Imagine que você foi escalado para um time de investimentos e precisará realizar um modelo preditivo com dados do IBOVESPA (Bolsa de Valores) para criar uma série temporal e prever diariamente o fechamento da base.

Para isso utilize a base  de dados contida no [Site da Investing](!https://br.investing.com/indices/bovespa-historical-data) e selecione o período diário, com o intervalo de tempo que achar adequado.

Você precisará demonstrar para o time de investimentos:

- O modelo com o storytelling, desde a captura do dado até a entrega do modelo;
- Justificar a técnica utilizada;
- Atingir uma acuracidade adequada (acima de 70%)

_Dica: utilize o maior intervalo de tempo possível para atingir maior acuracidade no modelo preditivo._

## Desenvolvimento




### Coleta e Tratamento dos dados

Consideraremos dados desde o primeiro dia do plano real (01/07/1994) em nossa análise preliminar, inclusive para entender a ocorrência de outliers desde a introdução de nossa moeda corrente.



In [69]:
import pandas as pd

df_ibovespa = pd.read_csv('/content/Dados Históricos - Ibovespa.csv')
df_ibovespa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      5000 non-null   object 
 1   Último    5000 non-null   float64
 2   Abertura  5000 non-null   float64
 3   Máxima    5000 non-null   float64
 4   Mínima    5000 non-null   float64
 5   Vol.      5000 non-null   object 
 6   Var%      5000 non-null   object 
dtypes: float64(4), object(3)
memory usage: 273.6+ KB


In [70]:
# Conferindo o resultado do info, vemos que os dados exportados não estão no formato ideal
# vamos fazer algumas conversões e conferir o cálculo da variação percentual
df_ibovespa['Data'] = pd.to_datetime(df_ibovespa['Data'], dayfirst=1)
df_ibovespa['Vol.'] = df_ibovespa['Vol.'].str.replace(',', '.')
def convert_value(value):
    if value[-1] == 'K':
        return int(float(value[:-1]) * 1000)
    elif value[-1] == 'M':
        return int(float(value[:-1])* 1000000)
    elif value[-1] == 'B':
        return int(float(value[:-1]) * 1000000000)
    else:
        return int(float(value))

df_ibovespa['Vol.'] = df_ibovespa['Vol.'].apply(convert_value)
df_ibovespa.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2014-09-12,56.928,58.336,58.336,56.666,4620000,"-2,42%"
1,2014-09-11,58.337,58.202,58.809,58.112,2890000,"0,24%"
2,2014-09-10,58.199,58.675,58.675,57.450,3810000,"-0,81%"
3,2014-09-09,58.676,59.158,59.485,58.306,4460000,"-0,87%"
4,2014-09-08,59.193,60.708,61.513,59.191,4390000,"-2,45%"


In [71]:
df_ibovespa['Var % Recalculada'] = ((df_ibovespa['Último'] / df_ibovespa['Abertura'] - 1) * 100).round(3)
df_ibovespa['Var%'] = df_ibovespa['Var%'].str.replace(',', '.').str.replace('%', '').astype(float)
df_ibovespa['Diff Var'] = ((df_ibovespa['Var%'] / df_ibovespa['Var % Recalculada'] - 1) * 100).round(1)
df_ibovespa.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Var % Recalculada,Diff Var
0,2014-09-12,56.928,58.336,58.336,56.666,4620000,-2.42,-2.414,0.2
1,2014-09-11,58.337,58.202,58.809,58.112,2890000,0.24,0.232,3.4
2,2014-09-10,58.199,58.675,58.675,57.450,3810000,-0.81,-0.811,-0.1
3,2014-09-09,58.676,59.158,59.485,58.306,4460000,-0.87,-0.815,6.7
4,2014-09-08,59.193,60.708,61.513,59.191,4390000,-2.45,-2.496,-1.8


In [72]:
# Conferindo a diferença com o valor recalculado podemos ver que há uma diferença
# significativa em alguns dias, portanto utilizaremos o valor recalculado como oficial
df_ibovespa.drop(columns=['Var%'], inplace=True)
df_ibovespa.drop(columns=['Diff Var'], inplace=True)
df_ibovespa.rename(columns={'Var % Recalculada': 'Var %'}, inplace=True)

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var %
0,2014-09-12,56.928,58.336,58.336,56.666,4620000,-2.414
1,2014-09-11,58.337,58.202,58.809,58.112,2890000,0.232
2,2014-09-10,58.199,58.675,58.675,57.450,3810000,-0.811
3,2014-09-09,58.676,59.158,59.485,58.306,4460000,-0.815
4,2014-09-08,59.193,60.708,61.513,59.191,4390000,-2.496


In [74]:
# dataframe já trabalhado
df_ibovespa.info()
df_ibovespa.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      5000 non-null   datetime64[ns]
 1   Último    5000 non-null   float64       
 2   Abertura  5000 non-null   float64       
 3   Máxima    5000 non-null   float64       
 4   Mínima    5000 non-null   float64       
 5   Vol.      5000 non-null   int64         
 6   Var %     5000 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 273.6 KB


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var %
0,2014-09-12,56.928,58.336,58.336,56.666,4620000,-2.414
1,2014-09-11,58.337,58.202,58.809,58.112,2890000,0.232
2,2014-09-10,58.199,58.675,58.675,57.450,3810000,-0.811
3,2014-09-09,58.676,59.158,59.485,58.306,4460000,-0.815
4,2014-09-08,59.193,60.708,61.513,59.191,4390000,-2.496


### Análise Exploratória

In [78]:
df_ibovespa.describe()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var %
count,5000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03,5000.000000
mean,2004-08-08 03:57:35.999999872,31.011401,31.002171,31.349051,30.653957,2.349664e+09,0.071056
min,1994-07-04 00:00:00,2.138000,2.138000,2.449000,2.110000,1.121000e+05,-15.779000
25%,1999-07-26 18:00:00,11.011250,11.014500,11.157500,10.850500,3.277500e+06,-1.024250
50%,2004-08-05 12:00:00,22.818500,22.791000,23.040500,22.506000,1.574000e+08,0.098500
75%,2009-08-24 06:00:00,54.057750,54.053750,54.629750,53.498250,6.834125e+08,1.220000
max,2014-09-12 00:00:00,73.517000,73.508000,73.920000,72.534000,5.406000e+10,33.896000
std,NaN,22.409299,22.409006,22.607527,22.201333,4.564622e+09,2.222023
